In [325]:
Query="What are all the education qualification of shyam?"

In [326]:
from dotenv import load_dotenv

load_dotenv()


True

In [327]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

def process_pdf_batch(pdf_file):
    loader = PyPDFLoader(pdf_file)
    pages = loader.load_and_split()
    character_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=0
        )
    pages = character_splitter.split_documents(pages)
    return pages

In [328]:
d="resume"

In [329]:
chunks=process_pdf_batch(f"data\{d}.pdf")

<string>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
<string>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
C:\Users\shyams\AppData\Local\Temp\ipykernel_14848\2212597078.py:1: SyntaxWarning: invalid escape sequence '\{'
  chunks=process_pdf_batch(f"data\{d}.pdf")


In [330]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
if not os.path.exists(f"{d}_index"):
    document_in_faiss = FAISS.from_documents(chunks, OpenAIEmbeddings())
    document_in_faiss.save_local(f"{d}_index")

In [331]:
document_in_faiss=FAISS.load_local(f"{d}_index", OpenAIEmbeddings(),allow_dangerous_deserialization=True)
faiss_retriever=document_in_faiss.as_retriever(search_kwargs={'k': 10})

In [332]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.load import dumps, loads

def generate_queries():
        
    # Multi Query: Different Perspectives
    template = """You are an AI language model assistant. Your task is to generate Four 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines. Original question: {question}"""
    prompt_perspectives = ChatPromptTemplate.from_template(template)


    generate_querie = (
        prompt_perspectives 
        | ChatOpenAI(temperature=0) 
        | StrOutputParser() 
        | (lambda x: x.split("\n"))
    )
    return generate_querie 


def _get_docs(m):
    docs=[]
    docs.extend(m['original'])
    for i in m['new']:
        docs.extend(i)
    return docs



In [333]:
from langchain_core.runnables import RunnableParallel
original_question= faiss_retriever
retrieval_chain =  generate_queries() | faiss_retriever.map()
map_chain = RunnableParallel(original=original_question,new=retrieval_chain) | _get_docs

In [302]:
# t=map_chain.invoke(Query)

In [303]:
# len(t)

20

In [304]:
# t

[Document(page_content='Shyam Sundar\n5,kavimani steet, Pankajam colony, 3rd cross street, Madurai, India - 625009\n♂phone+91-9080765574 /envel⌢pemailshyamsundar.2022@gmail.com /linkedinshyamsundar007 /githubShyam-Sundar-7♂laptopPortfolio\nProjects\nLLM-Powered Coupon Recommender /github/video |Python, Streamlit, Langchain, OpenAI November 2023\n•Developed a QA system for e-commerce with personalized coupon recommendations using OpenAI’s LLMs.\n•Streamlined user interactions through a Streamlit interface and Langchain for real-world scenario simulations.\n•Incorporated FAISS for refined recommendation processes.\nPeopleCare Insurance Prediction /github |Python, Jupyter, Azure Cloud, Flask, Docker October 2023\n•Expanded PeopleCare into vehicle insurance with a predictive model for effective customer targeting.\n•Thorough analysis of customer behavior and data cleaning for accurate predictive modeling.\n•Achieved 80% prediction accuracy using LightGBM.', metadata={'source': 'data\\resum

In [306]:
# t[0]

Document(page_content='Shyam Sundar\n5,kavimani steet, Pankajam colony, 3rd cross street, Madurai, India - 625009\n♂phone+91-9080765574 /envel⌢pemailshyamsundar.2022@gmail.com /linkedinshyamsundar007 /githubShyam-Sundar-7♂laptopPortfolio\nProjects\nLLM-Powered Coupon Recommender /github/video |Python, Streamlit, Langchain, OpenAI November 2023\n•Developed a QA system for e-commerce with personalized coupon recommendations using OpenAI’s LLMs.\n•Streamlined user interactions through a Streamlit interface and Langchain for real-world scenario simulations.\n•Incorporated FAISS for refined recommendation processes.\nPeopleCare Insurance Prediction /github |Python, Jupyter, Azure Cloud, Flask, Docker October 2023\n•Expanded PeopleCare into vehicle insurance with a predictive model for effective customer targeting.\n•Thorough analysis of customer behavior and data cleaning for accurate predictive modeling.\n•Achieved 80% prediction accuracy using LightGBM.', metadata={'source': 'data\\resume

# KEYWORDS BM25

In [334]:
def keyword_extractor():
    prompt="""
    You are an AI language model assistant. Your task is to help the user retrieve keywords from their query. 

    Please provide me with the keywords you would like to extract from your query. 

    Keywords: {keywords}
    """
    prompt_perspectives=ChatPromptTemplate.from_template(prompt)
    generate_querie = (
        prompt_perspectives 
        | ChatOpenAI(temperature=0) 
        | StrOutputParser() )
    return generate_querie

In [335]:
from langchain_community.retrievers import BM25Retriever
Bm25_retriever = BM25Retriever.from_documents(chunks)
Bm25_retriever.k = 10

In [336]:
from langchain.retrievers import EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[Bm25_retriever, map_chain], weights=[0.5, 0.5]
)

In [337]:
# final=ensemble_retriever.invoke(Query)

In [338]:
# final

[Document(page_content='Shyam Sundar\n5,kavimani steet, Pankajam colony, 3rd cross street, Madurai, India - 625009\n♂phone+91-9080765574 /envel⌢pemailshyamsundar.2022@gmail.com /linkedinshyamsundar007 /githubShyam-Sundar-7♂laptopPortfolio\nProjects\nLLM-Powered Coupon Recommender /github/video |Python, Streamlit, Langchain, OpenAI November 2023\n•Developed a QA system for e-commerce with personalized coupon recommendations using OpenAI’s LLMs.\n•Streamlined user interactions through a Streamlit interface and Langchain for real-world scenario simulations.\n•Incorporated FAISS for refined recommendation processes.\nPeopleCare Insurance Prediction /github |Python, Jupyter, Azure Cloud, Flask, Docker October 2023\n•Expanded PeopleCare into vehicle insurance with a predictive model for effective customer targeting.\n•Thorough analysis of customer behavior and data cleaning for accurate predictive modeling.\n•Achieved 80% prediction accuracy using LightGBM.', metadata={'source': 'data\\resum

In [339]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [344]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2")
compressor = CrossEncoderReranker(model=model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

# compressed_docs = compression_retriever.invoke(Query)
# pretty_print_docs(compressed_docs)

c:\Users\shyams\Downloads\RAG\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [341]:
final_prompt="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

final_prompt_perspectives=ChatPromptTemplate.from_template(final_prompt)

In [342]:
llm_chain = final_prompt_perspectives | ChatOpenAI(temperature=0) | StrOutputParser()

In [353]:
from operator import itemgetter
llm_chain2=({"context": itemgetter("query") | compression_retriever,
              "question":itemgetter("query")}
             | final_prompt_perspectives
             | ChatOpenAI(temperature=0) | StrOutputParser() )

In [354]:
llm_chain2.invoke({"query":Query})

'Shyam has a B.Tech in Chemical Engineering from the National Institute of Technology Tiruchirappalli with a GPA of 7.65 in May 2021 and an M.Tech in Modelling and Simulation from the Defence Institute of Advanced Technology Pune with a GPA of 7.95 in May 2023.'

In [1]:
!pip install chromadb

  Using cached chromadb-0.5.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached chroma-hnswlib-0.7.3.tar.gz (31 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.111.0-py3-none-any.whl.metadata (25 kB)
  Using cached uvicorn-0.29.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.17.3-cp312-cp312-win_amd64.whl.metadata (4.6 kB)
  Using cached opentelemetry_api-1.24.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.24.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_